<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 1**  
# **Диагностика и табличная репрезентация временных рядов: методы приведения к стационарности и инженерии признаков для задач прогнозирования**

---

## 1. Цель и задачи работы

**Цель работы** — формирование у обучающегося системного представления о конвейере предварительной обработки и диагностического анализа табличных данных с временной структурой, используемых в задачах машинного обучения для прогнозирования, освоение практических навыков сбора, очистки, визуализации и статистического анализа данных, а также приобретение компетенций в области подготовки репрезентативных датасетов, разработки интерактивных аналитических инструментов и публикации результатов исследования в соответствии с современными научно-методическими стандартами.

**Основные задачи работы:**  
1. Освоить методы сбора структурированных данных из открытых источников и сформировать репрезентативный табличный датасет с временной структурой, содержащий не менее трёх признаков и 720+ наблюдений.  
2. Реализовать программные модули предварительной очистки: обработка временных меток, пропусков, выбросов, приведение к единой временной зоне, удаление дубликатов.  
3. Провести описательный статистический анализ и визуализацию распределений, включая построение матрицы корреляций между признаками.  
4. Оценить стационарность целевой переменной с использованием формальных тестов (ADF, KPSS) и визуальных диагностик.  
5. Привести временной ряд к стационарности с применением лог-трансформации, преобразования Бокса–Кокса и дифференцирования (обычного и сезонного).  
6. Создать информативные временные признаки: лаги, скользящие статистики, сезонные и циклические кодировки.  
7. Провести анализ автокорреляционной структуры с помощью ACF и PACF для выявления значимых лагов.  
8. Выполнить аддитивную и мультипликативную декомпозицию целевой переменной и проанализировать свойства остатков.  
9. Разработать интерактивный веб-инструмент для визуального и функционального анализа табличных данных с временной структурой.  
10. Обеспечить воспроизводимость и открытость результатов: опубликовать датасет, программный код, веб-приложение и аналитический отчёт.

---

## 2. Теоретические предпосылки

В задачах прогнозирования машинное обучение оперирует табличными данными, где каждая строка соответствует наблюдению в определённый момент времени, а столбцы описывают состояние системы: целевую переменную, исторические значения, внешние факторы и контекст. Ключевой переход от исходного временного ряда к формату, пригодному для обучения моделей, заключается в создании признаков на основе прошлого поведения системы. Например, исходная запись:

```text
[timestamp, target, price, temperature]
```

преобразуется в расширенную табличную строку:

```text
[timestamp, target, price, temperature,
 target_lag_1, target_lag_7,
 rolling_mean_7, day_sin, day_cos, is_weekend, ...]
```

Такой подход позволяет применять алгоритмы машинного обучения, не обладающие встроенной памятью (например, градиентный бустинг), для моделирования временной зависимости.

Особое внимание уделяется **стационарности** — свойству ряда, при котором статистические характеристики (математическое ожидание, дисперсия, автокорреляция) не зависят от времени. Нестационарные ряды приводят к нестабильным и неинтерпретируемым моделям. Приведение к стационарности достигается через:
- **стабилизацию дисперсии** (лог-трансформация, Бокс–Кокс),
- **устранение тренда и сезонности** (дифференцирование),
- **декомпозицию** на компоненты.

Инженерия признаков для временных рядов включает не только лаги, но и **циклические кодировки** (например, `sin(day_of_week)`, `cos(day_of_week)`), которые сохраняют непрерывность временных меток и избегают ложной дискретности категориального кодирования.

Игнорирование этих аспектов приводит к неинформативным признакам, утечке данных и нестабильным прогнозам. Поэтому диагностический анализ является не подготовительным этапом, а неотъемлемой частью процесса построения прогнозных систем.

---

## 3. Порядок выполнения работы

Работа выполняется в несколько последовательных этапов, каждый из которых направлен на достижение конкретных образовательных и исследовательских результатов.

### 3.1. Формирование экспериментального табличного датасета с временной структурой

**Задача:** Сформировать многомерный датасет, репрезентативный для реальных прогнозных сценариев.  
**Требования к выполнению:**  
– Источники: Kaggle, Hugging Face Datasets, Росстат, ЦБ РФ, API (`yfinance`, `fredapi`, `openmeteo`) и др.;  
– Структура данных: колонка `timestamp` (в формате `datetime`), целевая переменная `target`, не менее двух дополнительных признаков (числовых или категориальных);  
– Минимальный объём: 720 последовательных наблюдений;  
– Формат хранения: `raw_dataset.csv` или `raw_dataset.parquet`;  
– Запрещено использование синтетических или чисто учебных временных рядов без внешних признаков.

### 3.2. Предварительная очистка и унификация данных

**Задача:** Привести датасет к аналитически пригодному виду.  
**Требования к выполнению:**  
– Приведение всех временных меток к единой временной зоне (например, `Europe/Moscow`);  
– Удаление дубликатов и проверка монотонности временной оси;  
– Обнаружение и коррекция выбросов на основе метода межквартильного размаха (IQR);  
– При необходимости — ресемплирование до регулярной частоты;  
– Код размещается в отдельном модуле `data_cleaner.py` с чётким разделением функциональности и документированием интерфейса.

### 3.3. Обработка пропущенных значений

**Задача:** Выявить и корректно обработать отсутствующие данные.  
**Требования к выполнению:**  
– Количественная оценка доли пропусков по каждому признаку;  
– Визуализация структуры пропусков (например, с помощью библиотеки `missingno`);  
– Применение метода заполнения в зависимости от контекста:  
  • линейная или полиномиальная интерполяция (для временных рядов);  
  • заполнение скользящим средним;  
  • прямое или обратное заполнение (`ffill`, `bfill`);  
  • удаление строк (только при доле пропусков менее 5% и при условии случайности пропусков);  
– Документирование принятых решений и их обоснование.

### 3.4. Инженерия признаков

**Задача:** Разработать конфигурируемый модуль для генерации информативных признаков.  
**Требования к выполнению:**  
– Генерация лаговых признаков (`target_lag_1`, `target_lag_7`, и др.);  
– Вычисление скользящих статистик (среднее, стандартное отклонение, минимум, максимум) с окнами 7, 30, 90;  
– Извлечение временных компонент (день недели, месяц, квартал, признак выходного дня);  
– **Циклическая кодировка** временных компонент:  
  • `day_sin = sin(2π ⋅ day_of_week / 7)`,  
  • `day_cos = cos(2π ⋅ day_of_week / 7)`,  
  • аналогично для месяца, часа суток и т.д.;  
– Создание взаимодействий признаков (например, произведение цены и рекламных расходов);  
– Расчёт относительных изменений (например, `target_diff_1 = target - target_lag_1`);  
– Поддержка загрузки правил из внешнего конфигурационного файла (YAML/JSON);  
– Реализация с использованием векторизованных операций `pandas` или `polars`.

### 3.5. Приведение временного ряда к стационарности и стабилизация дисперсии

**Задача:** Устранить нестационарность, вызванную трендом, сезонностью и нестабильной дисперсией.

**Требования к выполнению:**  
– **Проверка условий применимости преобразований:**  
  • Лог-трансформация и преобразование Бокса–Кокса применимы только при строго положительных значениях (`target > 0`). При необходимости — сдвиг ряда: `target_shifted = target + |min(target)| + ε`.  
– **Лог-трансформация:**  
  • Применяется для стабилизации экспоненциально растущей дисперсии;  
  • Формула: `log_target = np.log(target_shifted)`.  
– **Преобразование Бокса–Кокса:**  
  • Выполняется с автоматическим подбором оптимального параметра `λ` с использованием `scipy.stats.boxcox_normmax` (максимизация нормальности остатков);  
  • Сохраняется значение `λ` для последующего обратного преобразования прогнозов;  
  • Реализуется через `scipy.stats.boxcox`.  
– **Дифференцирование:**  
  • **Обычное (1-го порядка):** `diff_1 = target.diff(1)` — устраняет линейный тренд;  
  • **Сезонное:** `diff_s = target.diff(s)`, где `s` — период сезонности (например, 7 для дневных данных с недельной сезонностью);  
  • **Комбинированное:** последовательное применение, например: `target.diff(1).diff(7)`.  
– **Оценка эффективности преобразований:**  
  • После **каждого** преобразования проводится:  
    – Визуальный анализ (линейный график, скользящее среднее/дисперсия);  
    – Статистические тесты: **ADF** (H₀: нестационарность) и **KPSS** (H₀: стационарность);  
  • Фиксируются p-value, значения тестовых статистик и выводы.  
– **Выбор оптимальной цепочки преобразований:**  
  • Определяется последовательность, при которой ADF и KPSS согласуются в пользу стационарности (ADF: p < 0.05, KPSS: p > 0.05);  
  • Дисперсия стабилизирована, тренд и сезонность удалены.  
– **Обратимость:**  
  • Все преобразования должны допускать **обратное восстановление** исходного масштаба;  
  • Реализуется отдельный модуль `inverse_transform.py`.

> **Важно!** Обработка выбросов, пропусков и дубликатов **должна быть завершена до** применения преобразований Бокса–Кокса и дифференцирования.

### 3.6. Анализ автокорреляционной структуры

**Задача:** Оценить динамические свойства стационарного ряда.  
**Требования к выполнению:**  
– Построение и интерпретация графиков **ACF** (автокорреляционная функция) и **PACF** (частичная автокорреляционная функция);  
– Указание лагов, статистически значимых (выходящих за доверительный интервал ±1.96/√n);  
– Использование результатов для выбора порядков AR/MA компонент или значимых лагов в ML-моделях.

### 3.7. Декомпозиция временной компоненты и анализ остатков

**Задача:** Разделить целевую переменную на составляющие — чтобы понять, что «движет» данными.  
**Требования к выполнению:**  
– Применение декомпозиции:  
  • **аддитивной**: `value = trend + seasonality + residual`;  
  • **мультипликативной**: `value = trend × seasonality × residual` (при растущей амплитуде сезонности);  
– Визуализация каждой компоненты отдельно;  
– **Анализ остатков**:  
  • Проверка на стационарность (ADF/KPSS);  
  • Проверка на автокорреляцию (ACF остатков);  
  • Гистограмма и Q-Q график для оценки нормальности;  
  • Выявление остаточных паттернов (структурные разрывы, кластеры аномалий).

### 3.8. Разработка веб-интерфейса для анализа результатов

**Задача:** Создать интерактивное веб-приложение для визуального сравнения методов обработки данных.  
**Требования к выполнению:**  
– Возможность загрузки пользовательского CSV-файла или выбора из встроенных примеров;  
– Выбор целевой переменной и признаков для анализа;  
– Автоматическое построение графиков: временной ряд, ACF/PACF, heatmap корреляций, декомпозиция, эффект преобразований (до/после);  
– Формирование отчёта в формате HTML или PDF;  
– Рекомендуемые технологии: **Streamlit**, **Gradio** или фреймворки на основе **Flask/FastAPI** с библиотеками визуализации (**Plotly**, **Bokeh**).

### 3.9. Публикация результатов исследования

**Задача:** Обеспечить открытость и воспроизводимость работы.  
**Требования к выполнению:**  
– Для датасета и кода составляется метаописание, включающее:  
  • характеристики датасета (объём, источники, структура признаков);  
  • параметры анализа (лаги, окна, периоды, λ, порядок дифференцирования);  
  • ключевые метрики и выводы;  
  • ограничения и лицензионные условия;  
  • инструкцию по использованию;  
– Все материалы (датасет, код, веб-приложение, отчёт) публикуются в публичном репозитории (например, на **GitHub** или **GitLab**);  
– Развёртывание веб-приложения рекомендуется осуществлять на **Hugging Face Spaces**, **Render** или **Railway**;  
– Рекомендуется использование лицензий с открытым исходным кодом (**MIT**, **Apache 2.0**) и данных (**CC BY 4.0**).

---

## 4. Дополнительные исследовательские задания

1. Анализ кросс-корреляции (CCF) между целевой переменной и признаками — выявление временных лагов внешнего влияния.  
2. Проведение теста на структурные разрывы (CUSUM, Chow Test) — обнаружение точек смены режима динамики.  
3. Сравнение статистических свойств ряда при переходе между временными частотами (день → неделя → месяц).  
4. Анализ остатков после декомпозиции на стационарность и наличие автокорреляции.  
5. Экспериментальная оценка влияния периода сезонности на качество выделения тренда и остатков.  
6. Сравнение эффективности лог-трансформации и Бокса–Кокса по критерию нормальности остатков (Shapiro-Wilk, Jarque-Bera).

---

## 5. Требования к отчёту

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен содержать следующие разделы:  
1. Введение — постановка задачи, актуальность темы, обзор литературы.  
2. Методология — описание выбранных методов, инструментов, источников данных.  
3. Результаты экспериментов — таблицы, графики, статистические и количественные показатели (включая p-value ADF/KPSS, λ, порядки дифференцирования).  
4. Обсуждение — интерпретация результатов, выявление сильных и слабых сторон подходов, сравнение цепочек преобразований.  
5. Заключение — итоговые выводы и рекомендации.  
6. Список использованных источников — оформлен в соответствии с **ГОСТ Р 7.0.5–2008**.  
7. Приложения (при необходимости): фрагменты кода, скриншоты интерфейса, примеры датасета, схема конвейера обработки.

Отчёт сопровождается ссылками на репозиторий с кодом, веб-приложение и опубликованные данные.

---

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Полное выполнение всех этапов; наличие функционального веб-интерфейса; публикация датасета, кода и приложения; глубокий качественный и количественный анализ; отчёт, соответствующий ГОСТ; продемонстрирована обратимость преобразований и обоснован выбор цепочки стационаризации. |
| **Хорошо** | Выполнение всех основных этапов; корректный отчёт с визуализациями; базовый веб-инструмент; отсутствие лишь отдельных элементов публикации или анализа остатков. |
| **Удовлетворительно** | Реализованы этапы 3.1–3.5; отчёт содержит описание методов, результаты в табличной форме и обоснованные выводы; стационарность оценена, но без сравнения цепочек преобразований. |
| **Неудовлетворительно** | Отсутствуют ключевые компоненты работы (датасет, сравнительный анализ, отчёт) или работа не представлена. |
